In [ ]:
import pandas as pd
from apis_core.apis_metainfo.models import Collection as ACollection
from apis_core.apis_entities.models import Person as APerson
from apis_core.apis_entities.models import Place as APlace
from apis_core.apis_entities.models import Institution as AInstitution
from apis_core.apis_entities.models import Work as AWork

In [ ]:
file = "data/seh_works.csv"

In [ ]:
df = pd.read_csv(file, sep=';').fillna('False')

In [ ]:
resolver_uris = {
    "SigleHKA": "http://schnitzler.acdh.oeaw.ac.at/edition/api/resolve/work:{}",
    "FreiburgSystematik": "https://schnitzler.ub.uni-freiburg.de/work{}",
    "kolimo-schnitzlerID": "https://github.com/ingoboerner/kolimo-schnitzler/blob/master/Schnitzler_in_KOLIMO_meta.tsv#L{}",
    "HBAS": "https://bahrschnitzler.acdh.oeaw.ac.at/register.html?key={}",
    "wikidata": "http://www.wikidata.org/entity/{}",
}

In [ ]:
def make_uri(row, key, value):
    if row[key] != 'False':
        return value.format(row[key])
    else:
        return 'False'

In [ ]:
for key, value in resolver_uris.items():
    column_title = "{}-uri".format(key)
    df[column_title] = df.apply(lambda row: make_uri(row, key, value) , axis=1)

In [ ]:
df['new_name'] = df.apply(lambda row: "{} [frbr:Work]".format(row['Werktitel']), axis=1)

In [ ]:
col, _ = ACollection.objects.get_or_create(
    name='Worklist by Ingo'
)
basic_perswork_rel, _ = PersonWorkRelation.objects.get_or_create(
    name='bibliographical relation to work',
    name_reverse='bibliographical relation to person'
)
created, _ = PersonWorkRelation.objects.get_or_create(
    name='created',
    name_reverse='created_by',
    parent_class=basic_perswork_rel
)
schnitzler = APerson.objects.get(name='Schnitzler', first_name='Arthur')
work_kind, _ = WorkType.objects.get_or_create(name="frbr:Work")

In [ ]:
uris = [
    'HBAS-uri',
    'FreiburgSystematik-uri',
    'SigleHKA-uri',
    'wikidata-uri',
    'kolimo-schnitzlerID-uri',
    'theadok',
    'dracor'
]

In [ ]:
for i, row in df.iterrows():
    if row['new_name'] != 'False':
        work, _ = AWork.objects.get_or_create(
            name=row['new_name']
        )
        work.kind = work_kind
        for x in uris:
            if row[x] != 'False':
                try:
                    uri, _ = Uri.objects.get_or_create(
                    uri=row[x],
                    entity=work
                    )
                except:
                    pass
        PersonWork.objects.get_or_create(
                    relation_type=created,
                    related_person=schnitzler,
                    related_work=work
                )
        if row['ErstdruckHKA-Liste'] != 'False':
            work.start_date_written = str(int(row['ErstdruckHKA-Liste']))
        work.save()
        work.collection.add(col)